In [2]:
import gspread, numpy as np, pandas as pd

## setup suthorization to read/write googe sheets from colan
## requires one-time copy/paste auth code - follow link the appears when cell runs
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

class Sheet():
    def __init__(self, url, name=None):
        self.name = name  # name is optional
        self.url = url    # sheet url
        self.book = gc.open_by_url(url) # if permission error, change sharing setting in sheet using big "Share" button in top right corner

    def read(self, worksheet):
        """read data into pandas dataframe and do a litte datatype tweaks"""
        data = self.book.worksheet(worksheet).get_all_values()
        return pd.DataFrame(data=data[1:], columns=data[0]).apply(pd.to_numeric, errors='ignore', downcast='integer')

    def write(self, worksheet, df):
        """write df to worksheet; clears worksheet if exists or creates if not"""
        try:
            ws = self.book.add_worksheet(worksheet, *df.shape)
        except:
            ws = self.book.worksheet(worksheet)
            ws.clear()
        ws.update([df.columns.values.tolist()] + df.values.tolist())
        return ws

In [3]:
# open sheet in browser and copy/paste url in address bar below (enclosed within '' to make a python string)
# if permission error, change sharing setting in sheet using big "Share" button in top right corner
url = 'https://docs.google.com/spreadsheets/d/1tDfcrdIbBxq3nlQozhu6F6lb1nqynrtk9nU4fTO_IA0/edit#gid=0'
gs = Sheet(url)
df = gs.read('vartable')
df

,TableName,varNumber,varName,varTitle
0,HD2016,10186,ACT,Status of institution
1,ADM2016,10651,ACTCM25,ACT Composite 25th percentile score
2,ADM2016,10656,ACTCM75,ACT Composite 75th percentile score
3,ADM2016,10661,ACTEN25,ACT English 25th percentile score
4,ADM2016,10666,ACTEN75,ACT English 75th percentile score
...,...,...,...,...
2154,HD2016,16131,VETURL,Veterans and Military Servicemembers tuition p...
2155,HD2016,10081,WEBADDR,Institution's internet website address
2156,IC2016_PY,11471,WKCMP1,"Total length of program in weeks, as completed..."
2157,IC2016,10776,YRSCOLL,Years of college-level work required


In [4]:
# this should err b/c this sheet's share settings are "anyone with the link can VIEW"
gs.write('new', df)

WorksheetNotFound: ignored

# Exercise
Create your own google sheet, set share settings to "anyone with the link can EDIT", and write a pandas dataframe to it using gs.write